# C4W1 - 卷积神经网络
---
内容概要

1. 为什么需要卷积神经网络？
2. 卷积操作：filter、padding、stride等
3. 边缘检测
4. 池化：Max-Pooling 和 Average-Pooling

## 1 - 为什么需要卷积神经网络？

---

在Course1中，ng已经向我们展示了用标准神经网络（又称全连接神经网络）来处理图像分类任务，即识别图片中是否有猫。那么，为什么还需要卷积神经网络呢？

需要注意的是，Course1中处理的图片大小为 $64\times64\times3$，使用全连接神经网络，其第一个隐含层的权重参数只有12288个；如果图片大小为 $1000\times1000\times3$，那么权重参数将会是1百万个。也就是说，随着图片的增大，全连接神经网络的参数数量会急剧增多，这造成两个问题：1）神经网络的训练时长大幅上升；2）容易过拟合。

由此可见，当处理图像变大之后，使用全连接神经网络对其进行处理是行不通的，最主要的原因就是参数太多了。在卷积神经网络中，有两种方法可以大幅降低参数数量，分别是：1、权值共享（parameter sharing）；2、稀疏连接（sparsity of connections），又称局部感知。

Parameter sharing 中隐含的假设是图像中任一部分的统计特性与其他部分是一样的，这意味着，对图像中某一部分有效的特征检测器（feature detector）在图像的其他部分同样有效。

Sparsity of connections 的意思是，在网络中的每一层的每一个输出值仅依赖于图像输入的很小一部分区域。

---

## 2 - 卷积（convolutions）

---

典型的卷积过程如下图所示：
<img src="images/Conv2D.gif" style="width:200px">

上图中，黄色的$3\times3$矩阵为filter（过滤器，也有人称之为卷积核）。所谓卷积，就是filter在image矩阵上按照一定的规律移动、计算点积的过程。

关于点积，请查看：https://en.wikipedia.org/wiki/Dot_product

卷积运算的实现：
* tensorflow: tf.nn.conv2d
* keras: Conv2D

filter的大小f通常是奇数，这样做，有两个优势：
1. 方便padding操作
2. f是奇数的filter有一个中心点，便于计算，指出filter位置

---

### 2.1 - padding（填充）

---

使用 $f\times f$ 的filter在大小为 $n\times n$ 的矩阵上进行卷积操作，得到的结果矩阵会比原始矩阵小。但是，有的时候，我们并不希望结果矩阵比原始矩阵更小。此时，我们就需要对原始矩阵进行padding（填充）操作。

padding操作，其实就是在原始矩阵的四周填上合适数量的数值（通常是0），使得 $f\times f$ 的filter在矩阵上进行卷积操作之后能够得到与原始矩阵大小一样的结果矩阵。

使用padding的另外一个原因是：在没有经过padding的输入矩阵上进行卷积操作，矩阵边缘仅被卷积一次，这有可能造成信息丢失。

根据卷积过程是否进行了padding操作，卷积操作分为Valid convolutions（无padding）和Same convolutions（有padding）。


---


### 2.2 - stride（步幅）

---

步幅是控制filter在图像矩阵中某一维度移动跨度的度量。fliter可以在不同的维度有不同的stride，但是，通常将所有维度的步幅都设置成同样的大小。

### 2.3 -  边缘检测（edge detection）

---

图片中的边缘，最主要的一个特点是颜色骤变，在RGB矩阵的中体现是边缘两侧的数值相差较大。基于边缘的这个特性，设计一个合适的filter，将图像RGB矩阵中边缘两侧的数值转换成同样大小（通常转化为0），同时突出边缘两侧中间的数值，就能实现图像中的边缘检测。

<img src="images/edge_detect.png" style="width:500px">

上图是垂直边缘检测的一个简单例子，观察可以发现，原始输入矩阵有一条非常明显的垂直边缘，经过中间的filter卷积之后，在输出矩阵中就能看到边缘的位置了。


---

### 2.4 - Convolutions over volumes（立体卷积）

---

图片一般有RGB三个channels（通道），那么如何在图片上进行卷积操作呢？一个可行的方法是，将filter也设计成三通道，这样的话，RGB每个通道都有对应的filter矩阵，各通道计算点积之后，对应求和，就得到一个立体卷积的结果。示意图如下：

<img src="./images/conv3d.png" style="width:500px">

请注意，立体卷积之后，得到的结果是一个二维矩阵，也就是仅有一个channel，这很显然会丢失很多信息。为了使尽可能多的信息被保留下来，需要使用多个filter来进行卷积计算，需要多少个channels就使用多少个filter。使用多个filters的示意图如下：

<img src="./images/multi_filters.png" style="width:500px">


---

### 2.5 - 卷积过程总结

---

对于大小为 $n \times n$ 的图片，假设filter大小为 $f \times f$，padding参数为p，stride为s，则输出矩阵可以按如下公式计算：
<img src="./images/summary_conv.png" style="width:400px">

下面是从网上找到的一张gif图，展现了两个filter在图像（RGB三个通道）上的卷积过程，试着理解这张图：
<img src="./images/convolutions.gif" style="width:600px">

## 3 - Pooling（池化）

---

图像经过卷积层之后，输出矩阵往往比输入矩阵有所减小，但是依然很大。为了大幅减少参数的数量（也就是将权重系数矩阵减小），防止过拟合，通常会在卷积层后面加一个Pooling（池化）层。具体来说，池化层主要有两种：Max-Pooling（最大池化） 和 Average-Pooling（平均池化）。

池化过程和卷积过程非常相似，同样可以有 padding、stride等参数，不同的是，池化过程中的filter没有参数，只有大小。

以Max-Pooling为例，其过程如下图所示：

<img src="./images/pooling.jpg" style="width:400px">

从图中可以看出，其计算过程为：大小为 $ 2 \times 2$ 的filter以2个步幅在输入矩阵上进行移动，每次计算出filter范围内的最大值，作为输出。

Average-Pooling与Max-Pooling类似，不过，平均池化过程将filter范围内的平均值作为输出。


## 4 - 群内讨论总结

---

* 定义：分类=“图片里有什么”或”属于什么”，检测=“标定出特定的目标”；那么，原始CNN结构的分类能力强过检测能力
* 卷积操作对”平移不变性”有了良好的支持，但是对”旋转不变性”就无能为力了；“平移”指不旋转之外的上下左右移动,沿着斜线移动也可

## 参考资料

---

1. [梯度上升可视化卷积神经网络](https://mp.weixin.qq.com/s/tpfmh4PTMjFaMHKZLZw3Pg)
2. filter的解释：https://www.zhihu.com/question/52237725/answer/172479030
3. [在定义卷积时为什么要对其中一个函数进行翻转？](https://www.zhihu.com/question/20500497)